# Дообучение LLM в диалоговом режиме

## Установка зависимостей

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
!pip install torchmetrics evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.2 MB/s eta 0:00:00


In [ ]:
from torchmetrics.text import BLEUScore
import evaluate

## Загрузка модели

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B", "unsloth/Qwen2.5-32B", "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math, Coding versions!
    model_name = "unsloth/Qwen2.5-7B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

/tmp/ipython-input-3506749054.py:1: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.10: Fast Qwen2 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.16G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

## Добавление LoRA

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False, # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.8.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Подготовка данных

Формат Qwen-2.5 для диалоговогго дообучения
```
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
Hey there! How are you?<|im_end|>
<|im_start|>user
I'm great thanks!<|im_end|>
```



In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

def formatting_prompts_func(examples): # преобразует huggingface type к qwen type
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset

In [ ]:
dataset = load_dataset("igorktech/anekdots_dialogs")
dataset = dataset.remove_columns([
    'original', 'date', 'downvote', 'total_votes', 'upvote', 'hash',
    'alpha_frac', 'LDR', 'days_since_publication', 'time_decay', 'LDR_time_decay'
])

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/41.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100834 [00:00<?, ? examples/s]

Оставляем диалоги только с двумя высказываниями

In [ ]:
dataset = dataset["train"].filter(lambda row: len(row["parsed"]["turns"]) == 2)

Filter:   0%|          | 0/100834 [00:00<?, ? examples/s]

Сортировка датасета по убыванию оценки

In [ ]:
dataset = dataset.sort("total_mark", reverse=True)

In [ ]:
dataset = dataset.filter(lambda row, idx: idx < 30000, with_indices=True) # для малой модели нужно больше примеров, в случае LLM можно меньше

Filter:   0%|          | 0/48032 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['parsed', 'total_mark'],
    num_rows: 30000
})

## Преобразование данные в диалоговый формат huggingface



```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```



In [ ]:
def standartize(row): # chat_template работает именно в формате huggingface
    item = row["parsed"]
    row["conversations"] = [
        {'role': 'user', 'content': row['parsed']['turns'][0]['phrase']},
        {'role': 'assistant', 'content': row['parsed']['turns'][1]['phrase']}
    ]
    return row

In [ ]:
dataset = dataset.map(standartize, remove_columns=["parsed", "total_mark"])

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [ ]:
dataset[5]["conversations"]

[{'content': 'Доктор, мой ребенок наелся песка. Я дала ему много воды, а что теперь?',
  'role': 'user'},
 {'content': 'Следите, чтоб он не подходил к цементу...!!!',
  'role': 'assistant'}]

In [ ]:
dataset[5]["text"]

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nДоктор, мой ребенок наелся песка. Я дала ему много воды, а что теперь?<|im_end|>\n<|im_start|>assistant\nСледите, чтоб он не подходил к цементу...!!!<|im_end|>\n'

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
 dataset

DatasetDict({
    train: Dataset({
        features: ['conversations', 'text'],
        num_rows: 27000
    })
    test: Dataset({
        features: ['conversations', 'text'],
        num_rows: 3000
    })
})

## Настройка параметров модели

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,  # Can make training 5x faster for short sequences
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1,  # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # для quant моделей и под ограниченные ресурсы
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",  # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/27000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/3000 [00:00<?, ? examples/s]

## Обучение только на репликах второго спикера

In [ ]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only( # в этом методе по сути применяется маска к инструктивной части
    trainer,
    instruction_part = "<|im_start|>user\n", # инструкции которые хотим убрать
    response_part    = "<|im_start|>assistant\n",
)

Map (num_proc=2):   0%|          | 0/27000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nЧем новогодняя ночь будет отличаться от остальных?<|im_end|>\n<|im_start|>assistant\nЯ буду бухать в шапочке Деда Мороза.<|im_end|>\n'

In [ ]:
space = tokenizer(" ", add_special_tokens=False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                           Я буду бухать в шапочке Деда Мороза.<|im_end|>\n'

In [ ]:
eval_ds = dataset["test"].select(range(100))

## Тестовый запуск модели

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template( # токенизатор для qwen2.5
    tokenizer,
    chat_template = "qwen-2.5",
)

FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Сколько человек нужно, чтобы закрутить лампочку?"}, # пример huggingface type
]

inputs = tokenizer.apply_chat_template( # преобразование к формату qwen
    messages,
    tokenize = True,
    add_generation_prompt = True,  # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 64,
    use_cache = True,
    temperature = 1.5,
    min_p = 0.1
)

tokenizer.batch_decode(outputs)[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nСколько человек нужно, чтобы закрутить лампочку?<|im_end|>\n<|im_start|>assistant\nЧтобы закрутить лампочку, вам понадобится всего один человек. Другие могут помочь, если вы будете переживать из-за сломанной фитиля или проблем с доступом к лампочке, но technically一个人并不能拧灯泡'

### Вывод

1. Модель дала осмысленный ответ на вопрос. Выполнила роль ассистента.
2. Небольшие галлюцинации с "фитилём"
3. Генерирует длинный ответ
4. В конце какая-то смесь языков (??)

## Расчёт метрик BLEU и METEOR до обучения

Хранить в памяти две LLM дорого :) Поэтому для базовой LLM сразу расчитываем метрики

In [ ]:
from tqdm import tqdm

In [ ]:
from torchmetrics.text.bleu import BLEUScore

In [ ]:
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

bleu = BLEUScore(n_gram=1)
sum_init_bleu_score = 0.
sum_init_meteor_score = 0.

for row in tqdm(eval_ds):
    FastLanguageModel.for_inference(model)
    messages = [row["conversations"][0]]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                             temperature = 1.5, min_p = 0.1)
    pred = tokenizer.batch_decode(outputs)[0]
    target = row["text"]
    row["bleu"] = bleu(target, pred)

    metric_bleu = bleu(target, pred)
    sum_init_bleu_score += metric_bleu.item()

    metric_meteor = meteor.compute(predictions=[pred], references=[target])
    sum_init_meteor_score += metric_meteor["meteor"]

print('\navg bleu:', sum_init_bleu_score * 1. / len(eval_ds))
print('\navg meteor:', sum_init_meteor_score * 1. / len(eval_ds))

100%|██████████| 100/100 [07:15<00:00,  4.36s/it]


avg bleu: 0.7757278323173523

avg meteor: 0.6750852516822938


## Обучение

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.91 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 27,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Step,Training Loss,entropy
1,4.429100,0
2,5.404700,No Log
3,4.630100,No Log
4,5.270500,No Log
5,5.489000,No Log
6,4.170400,No Log
7,3.749200,No Log
8,3.159200,No Log
9,2.646500,No Log
10,2.394100,No Log


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

245.5829 seconds used for training.
4.09 minutes used for training.
Peak reserved memory = 7.992 GB.
Peak reserved memory for training = 0.082 GB.
Peak reserved memory % of max memory = 54.216 %.
Peak reserved memory for training % of max memory = 0.556 %.


## Тестовый запуск после обучения

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Сколько человек нужно, чтобы закрутить лампочку?"}, # пример huggingface type
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,  # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 64,
    use_cache = True,
    temperature = 1.5,
    min_p = 0.1
)

tokenizer.batch_decode(outputs)[0]

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nСколько человек нужно, чтобы закрутить лампочку?<|im_end|>\n<|im_start|>assistant\nНедостаточно людей - кого-то придется отрезать, а не только руки.<|im_end|>'

### Вывод

1. Ответ короче и не обрезается
2. Пытается шутить (темновато правда)

## Использование TextStreamer

In [ ]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Сколько человек нужно, чтобы закрутить лампочку?"}, # пример huggingface type
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,  # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer, skip_prompt = True)

_ = model.generate(
    input_ids = inputs,
    streamer = text_streamer,
    max_new_tokens = 128,
    use_cache = True,
    temperature = 1.5,
    min_p = 0.1
)

Все, кроме того, кого будут крутить!<|im_end|>


## Расчёт метрик BLEU и METEOR

In [ ]:
bleu = BLEUScore(n_gram=1)
sum_bleu_score = 0.
sum_meteor_score = 0.

for row in tqdm(eval_ds):
    FastLanguageModel.for_inference(model)
    messages = [row["conversations"][0]]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                             temperature = 1.5, min_p = 0.1)
    pred = tokenizer.batch_decode(outputs)[0]
    target = row["text"]
    row["bleu"] = bleu(target, pred)

    metric_bleu = bleu(target, pred)
    sum_bleu_score += metric_bleu.item()

    metric_meteor = meteor.compute(predictions=[pred], references=[target])
    sum_meteor_score += metric_meteor["meteor"]

print('\navg bleu:', sum_bleu_score * 1. / len(eval_ds))
print('\navg meteor:', sum_meteor_score * 1. / len(eval_ds))

100%|██████████| 100/100 [03:05<00:00,  1.85s/it]


avg bleu: 0.8184546303749084

avg meteor: 0.8231826735931955
